In [1]:
from dotenv import load_dotenv
import os
load_dotenv()
# from read_files import read_text_file
# from langchainutils import create_simple_chain
# from search import searchChunk, searchDocs
# from utils_from_aless import *
import tiktoken
from typing import AsyncIterable
from typing import List, Dict, Tuple, Optional, TypedDict
import re
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain.docstore.document import Document
from langchain.output_parsers import (
    OutputFixingParser,
)
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_community.chat_message_histories.in_memory import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.output_parsers import PydanticOutputParser
from azure.core import MatchConditions
import uuid
import random
from typing import Tuple, List, Dict
import json
from typing import Type, Optional, Dict
# from langchain_core.tools import CallbackManagerForToolRun, AsyncCallbackManagerForToolRun
# from search import searchChunk, searchDocs
from langchain.tools import BaseTool, StructuredTool, tool
import json
from langchain.pydantic_v1 import BaseModel, Field
from langchain.prompts import PromptTemplate, SystemMessagePromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate, ChatPromptTemplate
from langchain_core.prompts import format_document
from langchain.memory.entity import BaseEntityStore
from datetime import datetime, UTC
from langchain_core.runnables import RunnablePassthrough, Runnable
from typing import List, Dict, Tuple, Optional, AsyncIterable, Any, Mapping
from langchain_core.runnables.base import RunnableSerializable

from langchain_core.output_parsers.base import BaseOutputParser
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import Runnable
from typing import List, Type, Dict, TypeVar, Any, Union, Optional, NotRequired
from pathlib import Path
from functools import cache
import re

from enum import StrEnum
from langchain.prompts import PromptTemplate

from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings, OpenAI, ChatOpenAI
llm = ChatOpenAI(
    model_name="gpt-4o", openai_api_key=os.environ["OPENAI_API_KEY"], openai_organization=os.environ["OPENAI_ORGANIZATION"],
    temperature=0.3,
    streaming=True,
    request_timeout=60)

In [2]:
import os
import ast

def list_python_files(path):
    """
    List all Python files in the given directory path.
    """
    python_files = []
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith(".py"):
                python_files.append(os.path.join(root, file))
    return python_files


def get_functions_definitions(file_path):
    with open(file_path, 'r', encoding = 'utf-8') as file:
        source_code = file.read()
        tree = ast.parse(source_code)
        function_definitions = []

        for node in ast.walk(tree):
            if isinstance(node, ast.FunctionDef):
                function_code = ast.get_source_segment(source_code, node)
                function_definitions.append(function_code)

        return source_code

In [3]:
list_python_files("resources")

['resources\\IIRA\\app.py',
 'resources\\IIRA\\__init__.py',
 'resources\\IIRA\\core\\create_analyses.py',
 'resources\\IIRA\\core\\fileinteraction.py',
 'resources\\IIRA\\core\\metrics.py',
 'resources\\IIRA\\core\\__init__.py',
 'resources\\IIRA\\data\\__init__.py',
 'resources\\IIRA\\gui\\analyseframe.py',
 'resources\\IIRA\\gui\\containerframe.py',
 'resources\\IIRA\\gui\\fileframes.py',
 'resources\\IIRA\\gui\\helperframes.py',
 'resources\\IIRA\\gui\\mainframe.py',
 'resources\\IIRA\\gui\\rateframe.py',
 'resources\\IIRA\\gui\\__init__.py']

In [4]:
mainframe = get_functions_definitions("resources/IIRA/gui/mainframe.py")

In [5]:
function_prompt = """Can you write me a detailed docstrings for each function and classes you find in the python file in this path: {name}.
Here is the code: {contenu}.
Please just provide the function name or class name and the docstrings as such: {{'function_name': 'function.py', 'docstring': '..Initializes the main application window and its components..'}}, nothing else. 
The format to follow: {format_instructions}"""
# class_prompt = """Can you write me a detailed docstring for each classes you find here: {}. 
# Each of the functions of these classes are described by the docstrings: {}"""

def generate_function_doc(names: list[str], prompt:str) -> Dict:
    if not names:
        raise("You forgor bro")
    fonc_defs = []
    
    prompt1 = ChatPromptTemplate.from_messages(
        [
            ("system", "You are an expert in python coding."),
            ("human", prompt)
        ]
    )
    # Simple chain
    class docy(BaseModel):
        class_or_function: str = Field(description="This field can only be 'class' if it is a class or 'function' if it is a function.")
        class_of_the_function: str = Field(description="When it is a function, we need to know from which class it belongs to. Use the class name of the function.")
        function_name: str = Field(description="Name of the function or class found in the file.")
        docstring: str = Field(description="The very detailed docstring of the function or the class.")
    parser = JsonOutputParser(pydantic_object=docy)

    for name in names:
        prout = get_functions_definitions(name)
        if prout:
            fonc_defs.append({'name': name, 'contenu': prout, 'ai_generated': '', 'format_instructions': parser.get_format_instructions()})

    chain1 = prompt1 | llm | parser

    explains = []
    for dico in fonc_defs:
        explains.append(chain1.invoke(dico))  
    i = 0
    for elem in fonc_defs:
        elem['docstring'] = explains[i]
        i+=1
    return fonc_defs

In [8]:
coucou = generate_function_doc(list_python_files("resources"), function_prompt)

<unknown>:248: SyntaxWarning: invalid escape sequence '\.'


In [14]:
coucou[7]['docstring']

[{'class_or_function': 'class',
  'class_of_the_function': '',
  'function_name': 'ProfileFrame',
  'docstring': 'A class representing a profile management frame in a Tkinter Toplevel window. This frame allows users to create, delete, and switch profiles.'},
 {'class_or_function': 'function',
  'class_of_the_function': 'ProfileFrame',
  'function_name': '__init__',
  'docstring': 'Initializes the ProfileFrame with various widgets and layout configurations.\n\nArgs:\n    container (tk.Widget): The parent widget.'},
 {'class_or_function': 'function',
  'class_of_the_function': 'ProfileFrame',
  'function_name': 'ok_cmd',
  'docstring': 'Handles the OK button command. Creates a new profile if user input is provided, otherwise closes the window.\n\nArgs:\n    event (tk.Event, optional): The event that triggered the command. Defaults to None.'},
 {'class_or_function': 'function',
  'class_of_the_function': 'ProfileFrame',
  'function_name': 'populate_profile_label',
  'docstring': 'Updates 

In [10]:
who_are_you = """ 
Your task is to generate a readme file for an application. To give you context, i will give you the schema of the application by giving you the paths of each python file of the application. 
For each python file, I am providing you all its functions, classes and their docstrings.
{wooters}
Knowing all those informations please provide a detailed readme file for the entire application and nothing else.
"""

def i_am_still_him(objects: list[Dict], prompt:str) -> str:
    xd = ""
    
    # loop through whole objects (all .py files)
    for object in object_coucou:
        # loop through whole function or classes of each file
        for func in object['docstring']:
            # Function case
            if func['class_or_function'].casefold() == 'function'.casefold():

                inject_docstring(func['docstring'], app_py['name'], func['function_name'])
            if func['class_or_function'].casefold() == 'class'.casefold():
    
    for object in objects: 
        
    prompt1 = ChatPromptTemplate.from_messages(
        [("system", "You are an expert in python coding and especially good at writing 'readme' files."),
        ("human", prompt)])
    chain1 = prompt1 | llm | JsonOutputParser()
    return chain1.invoke({'wooters': xd})

In [9]:
import ast    
  
class DocstringInjector(ast.NodeTransformer):  
    def __init__(self, target_name, docstring, inject_to='class', indent='    '):  # Default indent is 4 spaces  
        self.target_name = target_name    
        self.docstring = docstring    
        self.inject_to = inject_to    
        self.indent = indent  
  
    def visit_ClassDef(self, node: ast.ClassDef):  
        if self.inject_to == 'class' and node.name == self.target_name and not ast.get_docstring(node):  
            indented_lines = [(self.indent + line) if line else '' for line in self.docstring.splitlines()]  
            indented_docstring = '\n'.join(indented_lines)  
            docstring_node = ast.Expr(value=ast.Constant(value=indented_docstring, kind=None))  
            node.body.insert(0, docstring_node)  
        self.generic_visit(node)  # Continue visiting child nodes  
        return node  
  
    def visit_FunctionDef(self, node: ast.FunctionDef):  
        if self.inject_to == 'function' and node.name == self.target_name and not ast.get_docstring(node):  
            indented_lines = [(self.indent + line) if line else '' for line in self.docstring.splitlines()]  
            indented_docstring = '\n'.join(indented_lines)  
            docstring_node = ast.Expr(value=ast.Constant(value=indented_docstring, kind=None))  
            node.body.insert(0, docstring_node)  
        self.generic_visit(node)  # Continue visiting child nodes  
        return node  
  
def inject_docstring(docstring, filename, target_name, inject_to='function'):  
    with open(filename, 'r') as file:  
        source = file.read()  
  
    tree = ast.parse(source)  
    injector = DocstringInjector(target_name=target_name, docstring=docstring, inject_to=inject_to)  
    modified_tree = injector.visit(tree)  
  
    modified_source = ast.unparse(modified_tree)  
  
    with open(filename, 'w') as file:  
        file.write(modified_source)  

docstring = '''This function does something.  
  
Parameters:  
    arg1 (type): Description of arg1  
    arg2 (type): Description of arg2  
  
Returns:  
    type: Description of return value  
'''  
inject_docstring(docstring, 'testingfile.py', 'FileFrame', 'class')  

AttributeError: 'Module' object has no attribute 'name'

In [16]:
coucou[0]

{'name': 'resources\\IIRA\\app.py',
 'contenu': '__author__ = "Timo Kubera"\n__email__ = "timo.kubera@stud.uni-hannover.de"\n\nimport os\nimport tkinter as tk\nfrom tkinter import ttk\n\nfrom gui.mainframe import MainFrame\nfrom gui.fileframes import FileFrame, ScaleFrame\nfrom gui.analyseframe import AnalyseFrame, ResultsFrame\nfrom gui.rateframe import RateFrame\nfrom core.fileinteraction import DBInteraction\n\nfrom PIL import ImageTk\n\nfile_path = os.path.dirname(os.path.realpath(__file__))\n\nclass App(tk.Tk):\n\n    def __init__(self):\n        super().__init__()\n        self.load_icons()\n\n        self.filevalidation = None\n        self.dbinteraction = DBInteraction(os.path.join(file_path, "data/internal_db.csv"))\n        self.scale_format = ""      # Skalenformate sind nominal, ordinal, intervall und ratio.\n        self.weights = ""\n        \n        self.categories = []\n        self.rater_ids = []\n        self.text = []\n        self.formatted_text = []\n        self.

In [28]:
coucou[0]['docstring'][0]['docstring']

'Main application class that inherits from tkinter.Tk. Initializes the main application window and its components, loads icons, and manages different frames.'

In [38]:
app_py = coucou[0]
for func in app_py['docstring']:
    if func['class_or_function'].casefold() == 'function'.casefold():
        # print(func['function_name'])
        # print(func['docstring'])
        # print(app_py['name'])
        # print(func['function_name'])
        inject_docstring(func['docstring'], app_py['name'], func['function_name'])  

AttributeError: 'DocstringInjector' object has no attribute 'class_name'

In [ ]:
def add_comments(object_coucou: Dict) -> str:
    # loop through whole objects (all .py files)
    for object in object_coucou:
        # loop through whole function or classes of each file
        for func in object['docstring']:
            # Function case
            if func['class_or_function'].casefold() == 'function'.casefold():

                inject_docstring(func['docstring'], app_py['name'], func['function_name'])
            if func['class_or_function'].casefold() == 'class'.casefold():
    return "success xd"